In [7]:
import time
import requests
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [20]:
driver = webdriver.Firefox()
driver.get("https://www.ucl.ac.uk/urban-lab/news")
time.sleep(10)# Allow 2 seconds for the web page to open
i = 0

urls = pd.DataFrame(columns=['title', 'url'])

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    links = soup.select('h3.heading.feed__item-heading>a')
    for link in links:
        urls.loc[len(urls), ['title', 'url']] = [link.text, link['href']]
    try:
        button = driver.find_element(By.CSS_SELECTOR, ".page-link.next")
        button.click()  
        i += 1
        time.sleep(1)
    except NoSuchElementException:
        break

In [22]:
urls.to_csv('./ucl-links.csv')

In [104]:
def fetch_article(url):
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    date = soup.select('strong.datepublished-article')[0].text.strip()
    paragraphs = soup.select('div.newsitemcontent.site-content__main>:is(p, ul, ol, a)')
    text = []
    for p in paragraphs:
        if p.find_all('li'):
            for li in p.find_all('li'):
                text.append(li.text.strip())
        else:
            text.append(p.text.strip())
    article = ' '.join(text)
    return [date, article]

In [106]:
corpus = pd.DataFrame(columns=['title', 'url', 'date', 'text'])

for i, row in tqdm(urls.iterrows()):
    title = row.title
    url = row.url
    date, text = fetch_article(url)
    corpus.loc[i, ['title', 'url', 'date', 'text']] = [title, url, date, text]

266it [04:13,  1.05it/s]


In [107]:
corpus

,title,url,date,text
0,Job opportunity: Project Co-ordinator,https://www.ucl.ac.uk/urban-lab/news/2022/apr/...,6 April 2022,Urban Laboratory are seeking to appoint a Proj...
1,Call for Papers: Deadline 24 April 2022,https://www.ucl.ac.uk/urban-lab/news/2022/apr/...,1 April 2022,"(Julie Livingston, Self-Devouring Growth, 2019..."
2,Seeking an Urban Room Curator,https://www.ucl.ac.uk/urban-lab/news/2022/mar/...,29 March 2022,The main purpose of the job is to curate and ...
3,UCL Urban Laboratory's MASc Global Urbanism op...,https://www.ucl.ac.uk/urban-lab/news/2022/mar/...,25 March 2022,This distinctive new MASc in Global Urbanism w...
4,Olympic Legacy Conference: Call for Contributions,https://www.ucl.ac.uk/urban-lab/news/2022/mar/...,3 March 2022,2022 marks the 10th anniversary of the London ...
...,...,...,...,...
261,Cities Methodologies Bucharest,https://www.ucl.ac.uk/urban-lab/news/2011/sep/...,12 September 2011,The publication is co-edited by Simona Dumitri...
262,David Roberts Offered UCL GRS Scholarship,https://www.ucl.ac.uk/urban-lab/news/2011/apr/...,4 April 2011,David Roberts' application for funding towards...
263,Second Annual Provost's Awards for Public Enga...,https://www.ucl.ac.uk/urban-lab/news/2011/jan/...,11 January 2011,"An urban food festival, a film competition abo..."
264,Stadtkolloquium-Urban PhD Research Seminar,https://www.ucl.ac.uk/urban-lab/news/2010/jan/...,28 January 2010,The workshop will provide an open and informal...


In [108]:
corpus.to_csv('ucl_corpus.csv')